# Data Processing - ADNI and demographics 

In [1]:
import pandas as pd
import regex as re
import numpy as np

In [2]:
path = 'C:/Users/glh19/OneDrive/Desktop/Winter_Spring 2022/Tosun-Turgut/data/'

adni1 = pd.read_csv(path + 'adni1_muse_results_with_dates.csv')
adni2 = pd.read_csv(path + 'adni2_muse_results_with_dates.csv')
adni3 = pd.read_csv(path + 'adni3_muse_results_with_dates.csv')

DATADIC = pd.read_csv(path + 'DATADIC.csv')
DXSUM = pd.read_csv(path + 'DXSUM_PDXCONV_ADNIALL.csv')
PTDEMOG = pd.read_csv(path + 'PTDEMOG.csv')

In [3]:
PTDEMOG_DXSUM = PTDEMOG.merge(DXSUM,  how='right')
PTDEMOG_DXSUM

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,PTSOURCE,PTGENDER,...,DXPCOG,DXPATYP,DXPOTHSP,DXDEP,DXDEPSP,DXOTHDEM,DXODES,DXOOTHSP,DXCONFID,DIAGNOSIS
0,ADNI1,2.0,2,107,bl,bl,2005-10-01,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN
1,ADNI1,336.0,2,107,m06,m06,2006-04-27,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,3.0,NaN
2,ADNI1,4.0,3,107,bl,bl,2005-10-01,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,3.0,NaN
3,ADNI1,338.0,3,107,m06,m06,2006-04-27,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN
4,ADNI1,990.0,3,107,m12,m12,2006-09-14,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12889,ADNI3,NaN,4556,41,y4,m114,2021-11-30,2021-11-30,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0
12890,ADNI3,NaN,7030,25,sc,sc,2021-11-30,2021-11-30,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0
12891,ADNI3,NaN,6025,40,y4,m48,2021-12-01,2021-12-01,NaN,NaN,...,NaN,NaN,NaN,1.0,scored 4 on GDS compared to 1 last time,NaN,NaN,NaN,NaN,1.0
12892,ADNI3,NaN,6604,50,y2,m24,2021-12-01,2021-12-01,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.0


In [4]:
def reorder_columns(dataframe, col_name, position):
    """Reorder a dataframe's column.
    Args:
        dataframe (pd.DataFrame): dataframe to use
        col_name (string): column name to move
        position (0-indexed position): where to relocate column to
    Returns:
        pd.DataFrame: re-assigned dataframe
    """
    temp_col = dataframe[col_name]
    dataframe = dataframe.drop(columns=[col_name])
    dataframe.insert(loc=position, column=col_name, value=temp_col)
    return dataframe

df = reorder_columns(dataframe=PTDEMOG_DXSUM, col_name='PTDOBMM', position=2)
df = reorder_columns(dataframe=df, col_name='PTDOBYY', position=3)


In [5]:
# slim_PTDEMOG_DXSUM = PTDEMOG_DXSUM.loc[:,['Phase','ID', 'RID','PTDOBMM','PTDOBYY','DXCHANGE','DXCURREN','DIAGNOSIS']]
keep_cols = ['Phase','ID', 'RID','PTDOBMM','PTDOBYY','DXCHANGE','DXCURREN','DIAGNOSIS']
remove_cols = ['SITEID','VISCODE','VISCODE2','USERDATE','USERDATE2',
 'PTSOURCE','PTGENDER','PTHAND','PTMARRY','PTEDUCAT','PTWORKHS',
 'PTWORK', 'PTWRECNT', 'PTNOTRT', 'PTRTYR', 'PTHOME', 'PTOTHOME',
 'PTTLANG', 'PTPLANG', 'PTPSPEC', 'PTCOGBEG', 'PTMCIBEG', 'PTADBEG',
 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'update_stamp','PTID','DXCONV',
 'DXCONTYP', 'DXREV', 'DXNORM', 'DXNODEP', 'DXMCI', 'DXMDES',
 'DXMPTR1', 'DXMPTR2', 'DXMPTR3', 'DXMPTR4', 'DXMPTR5', 'DXMPTR6',
 'DXMDUE', 'DXMOTHET', 'DXMOTHSP', 'DXDSEV', 'DXDDUE', 'DXAD',
 'DXADES', 'DXAPP', 'DXAPROB', 'DXAMETASP', 'DXAOTHRSP', 'DXAPOSS',
 'DXAATYSP', 'DXAMETSP', 'DXAOTHSP', 'DXPARK', 'DXPARKSP', 'DXPDES',
 'DXPCOG', 'DXPATYP', 'DXPOTHSP', 'DXDEP', 'DXDEPSP', 'DXOTHDEM',
 'DXODES', 'DXOOTHSP', 'DXCONFID']


new_order = ['Phase','ID', 'RID','PTDOBMM','PTDOBYY','DXCHANGE','DXCURREN','DIAGNOSIS',
 'SITEID','VISCODE','VISCODE2','USERDATE','USERDATE2',
 'PTSOURCE','PTGENDER','PTHAND','PTMARRY','PTEDUCAT','PTWORKHS',
 'PTWORK', 'PTWRECNT', 'PTNOTRT', 'PTRTYR', 'PTHOME', 'PTOTHOME',
 'PTTLANG', 'PTPLANG', 'PTPSPEC', 'PTCOGBEG', 'PTMCIBEG', 'PTADBEG',
 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'update_stamp','PTID','DXCONV',
 'DXCONTYP', 'DXREV', 'DXNORM', 'DXNODEP', 'DXMCI', 'DXMDES',
 'DXMPTR1', 'DXMPTR2', 'DXMPTR3', 'DXMPTR4', 'DXMPTR5', 'DXMPTR6',
 'DXMDUE', 'DXMOTHET', 'DXMOTHSP', 'DXDSEV', 'DXDDUE', 'DXAD',
 'DXADES', 'DXAPP', 'DXAPROB', 'DXAMETASP', 'DXAOTHRSP', 'DXAPOSS',
 'DXAATYSP', 'DXAMETSP', 'DXAOTHSP', 'DXPARK', 'DXPARKSP', 'DXPDES',
 'DXPCOG', 'DXPATYP', 'DXPOTHSP', 'DXDEP', 'DXDEPSP', 'DXOTHDEM',
 'DXODES', 'DXOOTHSP', 'DXCONFID']


# PTDEMOG_DXSUM.columns.tolist()

# PTDEMOG_DXSUM = PTDEMOG_DXSUM.reindex(columns=new_order)
# PTDEMOG_DXSUM
# slim_PTDEMOG_DXSUM = PTDEMOG_DXSUM.drop(remove_cols, axis=1, inplace=True)
# slim_PTDEMOG_DXSUM = PTDEMOG_DXSUM.drop(~PTDEMOG_DXSUM.isin(keep_cols), axis=1)

In [7]:
slim_PTDEMOG_DXSUM = PTDEMOG_DXSUM[['Phase','ID', 'RID','PTDOBMM','PTDOBYY','DXCHANGE','DXCURREN','DIAGNOSIS']]
slim_PTDEMOG_DXSUM.head(5)

,Phase,ID,RID,PTDOBMM,PTDOBYY,DXCHANGE,DXCURREN,DIAGNOSIS
0,ADNI1,2.0,2,NaN,NaN,NaN,1.0,NaN
1,ADNI1,336.0,2,NaN,NaN,NaN,1.0,NaN
2,ADNI1,4.0,3,NaN,NaN,NaN,3.0,NaN
3,ADNI1,338.0,3,NaN,NaN,NaN,3.0,NaN
4,ADNI1,990.0,3,NaN,NaN,NaN,3.0,NaN


In [8]:
#drop ADNIGO rows which originated from the DXSUM portion of dataframe
slim_PTDEMOG_DXSUM = slim_PTDEMOG_DXSUM[slim_PTDEMOG_DXSUM['Phase'] !='ADNIGO']
slim_PTDEMOG_DXSUM[slim_PTDEMOG_DXSUM['Phase']=='ADNIGO'].size

0

# Creating complete diagnosis col

In [9]:
#Success:
slim_PTDEMOG_DXSUM['CHAN_CURR'] = slim_PTDEMOG_DXSUM['DXCHANGE'].fillna(value=slim_PTDEMOG_DXSUM['DXCURREN'])
# slim_PTDEMOG_DXSUM['CHAN_CURR']
slim_PTDEMOG_DXSUM['complete_diagnosis'] = slim_PTDEMOG_DXSUM['CHAN_CURR'].fillna(value=slim_PTDEMOG_DXSUM['DIAGNOSIS'])

# slim_PTDEMOG_DXSUM[slim_PTDEMOG_DXSUM['complete_diagnosis']=='nan']
slim_PTDEMOG_DXSUM['complete_diagnosis']

0        1.0
1        1.0
2        3.0
3        3.0
4        3.0
        ... 
12889    1.0
12890    1.0
12891    1.0
12892    2.0
12893    1.0
Name: complete_diagnosis, Length: 12419, dtype: float64

In [10]:
adni12 = pd.concat([adni1,adni2], axis=0)
adni123 = pd.concat([adni12, adni3], axis=0)

# adni1.head(5)
# adni2.head(5)
# adni12
# adni12[adni12['project']=='ADNI2']
adni123[adni123['project']=='ADNI3']
# adni123

,project,code,roi_idx,roi_volume,roi_name,roi_index,examDate
0,ADNI3,ADNI3_002_S_0413y00_i863056,NaN,1.139552e+06,TOTALBRAIN,701.0,2017-06-21 13:27:00
1,ADNI3,ADNI3_002_S_0413y00_i863056,NaN,6.231692e+05,GM,601.0,2017-06-21 13:27:00
2,ADNI3,ADNI3_002_S_0413y00_i863056,NaN,4.512733e+05,WM,604.0,2017-06-21 13:27:00
3,ADNI3,ADNI3_002_S_0413y00_i863056,NaN,3.060864e+05,GM_L,606.0,2017-06-21 13:27:00
4,ADNI3,ADNI3_002_S_0413y00_i863056,NaN,2.179493e+05,WM_L,607.0,2017-06-21 13:27:00
...,...,...,...,...,...,...,...
359467,ADNI3,ADNI3_941_S_6854y01_i1428390,NaN,2.764000e+03,Right TrIFG triangular part of the inferior fr...,204.0,2021-04-05 00:00:00
359468,ADNI3,ADNI3_941_S_6854y01_i1428390,NaN,2.573000e+03,Left TrIFG triangular part of the inferior fro...,205.0,2021-04-05 00:00:00
359469,ADNI3,ADNI3_941_S_6854y01_i1428390,NaN,1.004000e+03,Right TTG transverse temporal gyrus,206.0,2021-04-05 00:00:00
359470,ADNI3,ADNI3_941_S_6854y01_i1428390,NaN,1.129000e+03,Left TTG transverse temporal gyrus,207.0,2021-04-05 00:00:00


# cleaning and removing unwanted roi volumes

In [11]:
# to remove: uppercase items
adni123.drop(adni123.index[adni123['roi_name'].str.isupper()], inplace=True)
print('adni123 size post uppercase removal:')
print(adni123.size)


adni123 size post uppercase removal:
4552940


In [12]:
discard = (['Right Cerebellum Exterior', 'Left Cerebellum Exterior',
            'Right Cerebellum White Matter','Left Cerebellum White Matter',
            'frontal lobe WM right','frontal lobe WM left',
            'occipital lobe WM right','occipital lobe WM left',
            'parietal lobe WM right','parietal lobe WM left',
            'temporal lobe WM right','temporal lobe WM left',
            'fornix right','fornix left','corpus callosum'])

# to remove unwanted brain regions
adni123 = adni123[adni123.roi_name.isin(discard) == False]
print('adni123 size post clean:')
print(adni123.size)

adni123 size post clean:
4096120


In [13]:
#remove unwanted ADNI1 partial string from code entries:
adni123['code'] = adni123['code'].str.strip('ADNI1_')
adni123['split_code'] = adni123['code'].str.split('_')
adni123.tail(5)

,project,code,roi_idx,roi_volume,roi_name,roi_index,examDate,split_code
359467,ADNI3,3_941_S_6854y01_i1428390,NaN,2764.0,Right TrIFG triangular part of the inferior fr...,204.0,2021-04-05 00:00:00,"[3, 941, S, 6854y01, i1428390]"
359468,ADNI3,3_941_S_6854y01_i1428390,NaN,2573.0,Left TrIFG triangular part of the inferior fro...,205.0,2021-04-05 00:00:00,"[3, 941, S, 6854y01, i1428390]"
359469,ADNI3,3_941_S_6854y01_i1428390,NaN,1004.0,Right TTG transverse temporal gyrus,206.0,2021-04-05 00:00:00,"[3, 941, S, 6854y01, i1428390]"
359470,ADNI3,3_941_S_6854y01_i1428390,NaN,1129.0,Left TTG transverse temporal gyrus,207.0,2021-04-05 00:00:00,"[3, 941, S, 6854y01, i1428390]"
359471,ADNI3,3_941_S_6854y01_i1428390,NaN,1652002.0,icv,NaN,2021-04-05 00:00:00,"[3, 941, S, 6854y01, i1428390]"


In [14]:
# split up code into new columns: PTID, RID, unique image #
# example code: 002_S_0295xia_118671

adni123['SITEID'] = adni123['split_code'].str[0]
adni123['RID_VISCODE'] = adni123['split_code'].str[2]
adni123['IMAGEUID'] = adni123['split_code'].str[3]

In [15]:
# RID=1326 and VISCODE=m12
rid_pattern = r'(\d\d\d\d)'
adni123['RID_VISCODE'] =  adni123['RID_VISCODE'].str.split(pat=rid_pattern)

# adni123['RID_VISCODE'].head(5)
adni123['RID'] = adni123['RID_VISCODE'].str[1]
adni123['VISCODE'] = adni123['RID_VISCODE'].str[2]

In [16]:
adni123[adni123['roi_name'] == 'icv'][['code','roi_name','roi_volume']]

,code,roi_name,roi_volume
776,002_S_0295X3a_64025,icv,1.854627e+06
1188,002_S_0295X6a_150177,icv,1.829614e+06
1447,002_S_0295X7a_291867,icv,1.785601e+06
1706,002_S_0413X1a_118673,icv,1.865068e+06
1965,002_S_0413X2a_118695,icv,1.870098e+06
...,...,...,...
358753,3_941_S_6580y01_i1226810,icv,1.455708e+06
359012,3_941_S_6581y00_i1048378,icv,1.777249e+06
359165,3_941_S_6581y02_i1343754,icv,1.713419e+06
359318,3_941_S_6854y00_i1291638,icv,1.641144e+06


# % icv column build

In [17]:
icv_values = (adni123[adni123['roi_name'] == 'icv']['roi_volume'])

adni123.groupby(['project','code'])['roi_volume'].size()

project  code                    
ADNI1    002_S_0295X1a_11867          28
         002_S_0295X2a_118692         28
         002_S_0295X3a_64025         130
         002_S_0295X6a_150177         40
         002_S_0295X7a_291867         40
                                    ... 
ADNI3    3_941_S_6580y01_i1226810    129
         3_941_S_6581y00_i1048378    129
         3_941_S_6581y02_i1343754    135
         3_941_S_6854y00_i1291638    135
         3_941_S_6854y01_i1428390    135
Name: roi_volume, Length: 6799, dtype: int64

In [18]:
icv_values

776       1.854627e+06
1188      1.829614e+06
1447      1.785601e+06
1706      1.865068e+06
1965      1.870098e+06
              ...     
358753    1.455708e+06
359012    1.777249e+06
359165    1.713419e+06
359318    1.641144e+06
359471    1.652002e+06
Name: roi_volume, Length: 4361, dtype: float64

In [19]:
adni123[(adni123.index.duplicated())&(adni123['project']=='ADNI1')]
adni123[adni123.index.duplicated()]

,project,code,roi_idx,roi_volume,roi_name,roi_index,examDate,split_code,SITEID,RID_VISCODE,IMAGEUID,RID,VISCODE
113,ADNI2,2_002_S_0295v06_i241350,NaN,3.735600e+03,Left PCgG posterior cingulate gyrus,167.0,6/2/2011,"[2, 002, S, 0295v06, i241350]",2,[S],0295v06,NaN,NaN
114,ADNI2,2_002_S_0295v06_i241350,NaN,1.148520e+04,Right PCu precuneus,168.0,6/2/2011,"[2, 002, S, 0295v06, i241350]",2,[S],0295v06,NaN,NaN
115,ADNI2,2_002_S_0295v06_i241350,NaN,1.174800e+04,Left PCu precuneus,169.0,6/2/2011,"[2, 002, S, 0295v06, i241350]",2,[S],0295v06,NaN,NaN
116,ADNI2,2_002_S_0295v06_i241350,NaN,3.229200e+03,Right PHG parahippocampal gyrus,170.0,6/2/2011,"[2, 002, S, 0295v06, i241350]",2,[S],0295v06,NaN,NaN
117,ADNI2,2_002_S_0295v06_i241350,NaN,3.438000e+03,Left PHG parahippocampal gyrus,171.0,6/2/2011,"[2, 002, S, 0295v06, i241350]",2,[S],0295v06,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
359467,ADNI3,3_941_S_6854y01_i1428390,NaN,2.764000e+03,Right TrIFG triangular part of the inferior fr...,204.0,2021-04-05 00:00:00,"[3, 941, S, 6854y01, i1428390]",3,[S],6854y01,NaN,NaN
359468,ADNI3,3_941_S_6854y01_i1428390,NaN,2.573000e+03,Left TrIFG triangular part of the inferior fro...,205.0,2021-04-05 00:00:00,"[3, 941, S, 6854y01, i1428390]",3,[S],6854y01,NaN,NaN
359469,ADNI3,3_941_S_6854y01_i1428390,NaN,1.004000e+03,Right TTG transverse temporal gyrus,206.0,2021-04-05 00:00:00,"[3, 941, S, 6854y01, i1428390]",3,[S],6854y01,NaN,NaN
359470,ADNI3,3_941_S_6854y01_i1428390,NaN,1.129000e+03,Left TTG transverse temporal gyrus,207.0,2021-04-05 00:00:00,"[3, 941, S, 6854y01, i1428390]",3,[S],6854y01,NaN,NaN


In [22]:
# reindex adni123
adni123 = adni123.reset_index() 

In [23]:
adni123['icv_vals'] = adni123[adni123['roi_name']=='icv']['roi_volume']
adni123['icv_vals']

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
            ...    
585155          NaN
585156          NaN
585157          NaN
585158          NaN
585159    1652002.0
Name: icv_vals, Length: 585160, dtype: float64

In [24]:
adni123['icv_vals'] = adni123['icv_vals'].fillna(0)


In [ ]:
def divide_two_cols(adni123):
    adni123['%_icv'] = adni123['roi_volume'] / adni123['icv_vals'].max()
    return adni123['%_icv']

# I might have to groupby 'code' and 'project' b/c the same patient may be in multiple phases with different brain vol measurements 
# checking if function filtered data properly when computing % icv:

adni123['%_icv'] = adni123.groupby(['code','project']).apply(divide_two_cols).tolist()

# adni123[adni123['code']=='016_S_1326m12_i102675'][['code','roi_name','roi_volume','%_icv']]

adni123[((adni123['roi_name']=='icv') | (adni123['roi_name']== 'Left Amygdala'))&(adni123['project']=='ADNI2')][['project','code','roi_name','roi_volume','%_icv']]


# adni123['%_icv'] = adni123.groupby('code').apply(divide_two_cols).tolist()

In [ ]:
adni123[adni123['code']=='016_S_1326m12_i102675'][['code','roi_name','roi_volume','%_icv']]

In [ ]:
# final merge:
adni_merged = slim_PTDEMOG_DXSUM.merge(adni123, on='RID', how='left')
# adni_merged = pd.concat(adni123,slim_PTDEMOG_DXSUM)

adni_merged

# Calculating age 

In [21]:
from datetime import datetime as dt
from datetime import timedelta

#using fake data until I am able to successfully acces the mm and yy columns
# of the dataframe while taking into account the indexing and formatting

age_practice = pd.DataFrame(data= {
    'PTDOBMM':[4,12,5,1],
    'PTDOBYY':[1931,1944,1924,1938],
    'examDate':['4/18/2006','11/2/2006','7/23/2008','5/22/2009']
    
})

age_practice['defaultDD'] = 15*np.ones(age_practice.shape[0]).astype(int)
#columns to make date: from month, day, year:
cols = ["PTDOBMM","defaultDD","PTDOBYY"]
age_practice["DOB"] = age_practice[cols].apply(lambda x: '/'.join(x.values.astype(str)), axis="columns")
# age_practice["DOB"] = age_practice["DOB"].str.split('/').str[0]

#transform to datetime format:
age_practice['examDate'] = age_practice['examDate'].str.split('/').str[2]+'-'+age_practice['examDate'].str.split('/').str[0]+'-'+age_practice['examDate'].str.split('/').str[1]
age_practice['DOB'] = age_practice['DOB'].str.split('/').str[2]+'-'+age_practice['DOB'].str.split('/').str[0]+'-'+age_practice['DOB'].str.split('/').str[1]

age_practice['examDate'] = pd.to_datetime(age_practice['examDate'],format='%Y-%m-%d')

age_practice['DOB'] = pd.to_datetime(age_practice['DOB'],format='%Y-%m-%d')


# compute age:
age_practice['age_in_days'] = age_practice['examDate'] -age_practice['DOB']

age_practice['age_in_years'] = age_practice["age_in_days"] / timedelta(days=365)


age_practice

,PTDOBMM,PTDOBYY,examDate,defaultDD,DOB,age_in_days,age_in_years
0,4,1931,2006-04-18,15,1931-04-15,27397 days,75.060274
1,12,1944,2006-11-02,15,1944-12-15,22602 days,61.923288
2,5,1924,2008-07-23,15,1924-05-15,30750 days,84.246575
3,1,1938,2009-05-22,15,1938-01-15,26060 days,71.397260
